# Live demo

In this notebook, we will use model we trained to move jetBot smoothly on track. 

### Load Trained Model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` and ``best_model.pth`` to work station as instructed in "collision_avoidance/train_model.ipynb" and "Road_Following/train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebooks's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` and ``best_model.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [ ]:
import torchvision
import torch

# road following 
model_road_following = torchvision.models.resnet18(pretrained=False)
model_road_following.fc = torch.nn.Linear(512, 2)

# collision avoidance
item = ['free' , 'green_light' , 'red_light' , 'parking' , 'zebra' , 'fence_open' , 'fence_close' , 'duck']
model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, len(item))

Next, load the trained weights from the ``best_steering_model_xy.pth`` and ``best_model.pth`` file that you uploaded.

In [ ]:
# road following
model_road_following.load_state_dict(torch.load('best_steering_model_xy.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [ ]:
device = torch.device('cuda')

# collision avoidance
model_collision = model_collision.to(device)

### Creating the Pre-Processing Function

For road following :
We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps:

1. Convert from HWC layout to CHW layout
2. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
3. Transfer the data from CPU memory to GPU memory
4. Add a batch dimension

For collision avoidance :
We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2 , time
import PIL.Image
import numpy as np

# road following
road_following_mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
road_following_std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def road_following_preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(road_following_mean[:, None, None]).div_(road_following_std[:, None, None])
    return image[None, ...]


# collision avoidance
collision_mean = 255.0 * np.array([0.485, 0.456, 0.406])
collision_stdev = 255.0 * np.array([0.229, 0.224, 0.225])

collision_normalize = torchvision.transforms.Normalize(collision_mean, collision_stdev)

def collision_preprocess(camera_value):
    global device, collision_normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = collision_normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Awesome! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

In [ ]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

## configure cameara :  Camera.instance(capture_flip=2,width=224, height=224)
camera = Camera.instance(capture_flip=2, width=224, height=224)


In [ ]:
image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

#traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

slider = {}

for text in item:
    slider[text] = ipywidgets.widgets.FloatSlider(description=text, min=0.0, max=1.0, orientation='vertical')

# If you have four serval items to do , please type :  display(widgets.HBox([image, slider[item[0]] , slider[item[1]] , slider[item[2]] , slider[item[3]] , ... ])) )
# notice the start number is 0 .
display(ipywidgets.widgets.HBox([image_widget, slider[item[0]] , slider[item[1]] , slider[item[2]] , slider[item[3]] , slider[item[4]] , slider[item[5]] , slider[item[6]] , slider[item[7]] ]))

Next, we'll create a function that will get called whenever the camera's value changes. This function will do the following steps

For road following : 

1. Pre-process the camera image
2. Execute the neural network
3. Compute the approximate steering value
4. Control the motors using proportional / derivative control (PD)


For collision avoidance :

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [ ]:
def execute(change):
    global  slider 
    image = change['new']
    # collision avoidance
    collision_x = collision_preprocess(image)
    collision_y = model_collision(collision_x)

    ## we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    collision_y = F.softmax(collision_y, dim=1)
    confidence = {}
    # item = ['free' , 'green_light' , 'red_light' , 'parking' , 'zebra' , 'fence_open' , 'fence_close' , 'duck']
    for text in item:
        confidence[text] = float(collision_y.flatten()[item.index(text)])
        slider[text].value = confidence[text]
        
execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [ ]:
camera.observe(execute, names='value')

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame. 

You can now place JetBot on  Lego or Track you have collected data on and see whether it can follow track.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
camera.unobserve(execute, names='value')

### Conclusion
That's it for this live demo! Hopefully you had some fun seeing your JetBot moving smoothly on track follwing the road!!!

If your JetBot wasn't following road very well, try to spot where it fails. The beauty is that we can collect more data for these failure scenarios and the JetBot should get even better :)